In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
URL = "https://www.tirabeauty.com/collection/makeup-lip?brand=chambor&sort_on=price_asc"

In [3]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

In [4]:
webpage = requests.get(URL, headers=HEADERS)

In [5]:
type(webpage.content)

bytes

In [6]:
webpage

<Response [200]>

In [7]:
soup = BeautifulSoup(webpage.content, "html.parser")

In [8]:
soup

<!DOCTYPE html>

<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D,%22style%22:%7B%22ssr%22:%22--primaryColor:#030303;--secondaryColor:#511778;%20--accentColor:#01010e;--linkColor:#f00000;--bgColor:#ffffff;--pageBackground:#ffffff;--themeAccent:#01010e;--textHeading:#511778;--textBody:#3C3131;--textLabel:#7D7676;--textSecondary:#9C9C9C;--buttonPrimary:#030303;--buttonSecondary:#000000;--buttonLink:#f00000;--saleBadgeBackground:#FFFFFF;--saleBadgeText:#1C958F;--saleDiscountText:#1C958F;--saleTimer:#994449;--headerBackground:#F3F3ED;--headerNav:#261A1A;--headerIcon:#261A1A;--footerBackground:#2C231E;--footerBottomBackground:#231812;--footerHeadingText:#FFFFFF;--footerBodyText:#FFFFFF;--footerIcon:#FFFFFF;--dialogBackground:#FFFFFF;--overlay:#14130E;--dividerStokes:#D4D1D1;--highlightColor:#EDECE9;--successBackground:#C2DBC9;--successText:#1C958F;--errorBackground:#E6D5D5;--errorText:#B24141;--informationBackground:#EBD3BC;--informationText:#D28F51;%22%7D%7D" lang="en" style="--

In [9]:
import sqlite3

# Extract data
product_names = [product.get_text(strip=True) for product in soup.find_all('div', class_='product-name')]
actual_prices = [price.get_text(strip=True) for price in soup.find_all('p', class_='actual-price')]
discount_prices = [newprice.get_text(strip=True) for newprice in soup.find_all('p', class_='discount-price')]
discount_percentages = [discount.get_text(strip=True) for discount in soup.find_all('p', class_='discount-percentage')]

# Connect to SQLite database
conn = sqlite3.connect('lipstick.db')
cursor = conn.cursor()

# Create the 'lipstick' table
cursor.execute('''
CREATE TABLE IF NOT EXISTS lipstick (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT,
    actual_price TEXT,
    discount_price TEXT,
    discount_percentage TEXT
)
''')

# Insert data into the 'lipstick' table
for name, actual_price, discount_price, discount_percentage in zip(product_names, actual_prices, discount_prices, discount_percentages):
    cursor.execute('''
    INSERT INTO lipstick (product_name, actual_price, discount_price, discount_percentage)
    VALUES (?, ?, ?, ?)
    ''', (name, actual_price, discount_price, discount_percentage))

# Commit the changes and close the connection
conn.commit()
conn.close()